In [13]:
import os
import shutil

# Create directories for categories
os.makedirs('train/cat', exist_ok=True)
os.makedirs('train/dog', exist_ok=True)

# Directory where the images are initially stored
source_dir = 'train'

# Process each file in the source directory
for filename in os.listdir(source_dir):
    if filename.endswith(('.jpg', '.jpeg', '.png')):  # Check for image files
        if filename.startswith('cat'):
            category = 'cat'
        else:
            category = 'dog'
        
        # Source file path
        source_path = os.path.join(source_dir, filename)
        
        # Destination file path
        destination_path = os.path.join(source_dir, category, filename)
        
        # Move the file to the appropriate category directory
        shutil.move(source_path, destination_path)

print("Files have been successfully moved to their respective directories.")


Files have been successfully moved to their respective directories.


In [47]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

train_dir = 'train'
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((128,128))
])
train_dataset = ImageFolder(root = train_dir,
                            transform = transform)
train_dataloader = DataLoader(train_dataset, 
                              shuffle = True,
                              batch_size = 1)


In [48]:
img_batch, label_batch  = next(iter(train_dataloader))
print(img_batch, img_batch.shape)
print(label_batch, label_batch.shape)

tensor([[[[1.4183e-02, 1.0403e-02, 1.0333e-02,  ..., 7.8985e-01,
           9.0956e-01, 9.1440e-01],
          [1.1037e-02, 1.1341e-02, 7.8113e-03,  ..., 8.2377e-01,
           9.1702e-01, 9.1517e-01],
          [1.4810e-02, 7.3012e-03, 6.8339e-03,  ..., 8.7358e-01,
           9.2442e-01, 9.2425e-01],
          ...,
          [6.3807e-01, 6.4623e-01, 6.6264e-01,  ..., 2.2673e-01,
           2.3473e-01, 2.2335e-01],
          [6.4038e-01, 6.5977e-01, 6.4710e-01,  ..., 2.1819e-01,
           2.2576e-01, 2.2538e-01],
          [6.4367e-01, 6.6422e-01, 6.5157e-01,  ..., 2.3738e-01,
           2.2634e-01, 2.2472e-01]],

         [[6.6405e-03, 3.3806e-03, 2.9737e-03,  ..., 7.5566e-01,
           8.8069e-01, 8.7204e-01],
          [3.9130e-03, 4.2027e-03, 1.5302e-03,  ..., 7.7956e-01,
           8.8302e-01, 8.8186e-01],
          [7.0527e-03, 1.0767e-03, 8.6437e-06,  ..., 8.1955e-01,
           8.8416e-01, 8.9660e-01],
          ...,
          [6.4199e-01, 6.5015e-01, 6.6656e-01,  ..., 2.2049

In [49]:
import torch.nn as nn

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(3,32, kernel_size = 3, padding = 1),
            nn.ELU(),
            nn.MaxPool2d(kernel_size = 2),
            nn.Conv2d(32, 64, kernel_size = 3, padding = 1),
            nn.ELU(),
            nn.MaxPool2d(kernel_size = 2),
            nn.Flatten(),
        )
        self.classifier = nn.Linear(64*32*32, 1)
                    
    def forward(self, x):
        x = self.feature_extractor(x)
        x = self.classifier(x)
        return x

In [52]:
import torch
import torch.optim as optim
net = Net()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(net.parameters(), lr = 0.001)

for epoch in range(1):
    for features, labels in train_dataloader:
        optimizer.zero_grad()
        outputs = net(features)
        loss = criterion(outputs, labels.view(-1,1).float())
        loss.backward()
        optimizer.step()


In [64]:
torch.save(net.state_dict(), 'torchmodel.pth')

In [66]:
from PIL import Image
image = Image.open('cat.1.jpg')
image_tensor = transform(image)
image_re = image_tensor.unsqueeze(0)
out = net(image_re)
prob = torch.sigmoid(out)
prob

tensor([[0.4033]], grad_fn=<SigmoidBackward0>)

In [ ]:
import torch
import torch.nn as nn

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(3,32, kernel_size = 3, padding = 1),
            nn.ELU(),
            nn.MaxPool2d(kernel_size = 2),
            nn.Conv2d(32, 64, kernel_size = 3, padding = 1),
            nn.ELU(),
            nn.MaxPool2d(kernel_size = 2),
            nn.Flatten(),
        )
        self.classifier = nn.Linear(64*32*32, 1)
                    
    def forward(self, x):
        x = self.feature_extractor(x)
        x = self.classifier(x)
        return x
    
model = Net()
model.load_state_dict(torch.load('model_torch.pth'))

model
import os
folder_path = "test"
image_paths = []
for image_name in os.listdir(folder_path):
    image_path = os.path.join(folder_path, image_name)
    image_paths.append(image_path)
from torchvision.models import vgg16, VGG16_Weights

vgg = vgg16(weights = VGG16_Weights.DEFAULT)
backbone = nn.Sequential(
    *list(vgg.features.children())
)

import torch.nn as nn

class BinaryCNN(nn.Module):
    def __init__(self):
        super(BinaryCNN, self).__init__()
        self.backbone = nn.Sequential(*list(vgg.features.children()))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(25088, 2000),
            nn.ELU(),
            nn.Linear(2000,1),
            nn.Sigmoid()
        )
                    
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x
BinaryCNN()
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

train_dir = 'train'
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224,224))
])
train_dataset = ImageFolder(root = train_dir,
                            transform = transform)
train_dataloader = DataLoader(train_dataset, 
                              shuffle = True,
                              batch_size = 1)

img,_ = next(iter(train_dataloader))
img.shape
import torch
import torch.nn as nn
import torch.optim as optim

# Assuming `BinaryCNN` class definition here...

net = BinaryCNN()
criterion = nn.BCELoss()

# Freeze backbone parameters
for param in net.backbone.parameters():
    param.requires_grad = False

optimizer = optim.SGD(filter(lambda p: p.requires_grad, net.parameters()), lr=0.001)

for epoch in range(1):
    for features, labels in train_dataloader:
        optimizer.zero_grad()
        outputs = net(features)
        loss = criterion(outputs, labels.view(-1, 1).float())
        loss.backward()
        optimizer.step()

from PIL import Image
image = Image.open('test2.jpg')
image_tensor = transform(image)
image_re = image_tensor.unsqueeze(0)
out = net(image_re)
prob = torch.sigmoid(out)
prob